In [1]:
import pandas as pd 
import numpy as np
import os, sys, csv

In [2]:
src_df = {}
src_files = os.scandir('src_forms2')

In [3]:
for i in src_files:
    src_df[i.name.split('.')[0]]  = pd.read_csv(i.path)

In [4]:
src_df['forms_raw4_2'] = src_df['forms_raw4_2'].drop_duplicates()

In [5]:
def create_unit_key(x):
    x = x.dropna()
    if 'label' in x and 'value' in x:
        return ','.join([x['label'], x['value'], 'Нет'])
    elif 'label' in x and 'options' in x:
        x['options'] = x['options'].split('\\r\\n')
        for y in range(len(x['options'])):
            x['options'][y] = x['options'][y].split('=')[0]
        x['options'] = ','.join(x['options'])
        return ','.join([x['label'], x['options']])
    elif 'label' in x:
        return x['label']
    else :
        return ','.join(x.dropna().astype(str))

In [6]:
src_df['forms_raw4_2']['unitkey'] = src_df['forms_raw4_2'].apply(lambda x: create_unit_key(x) ,axis=1)

In [7]:
src_df['forms_name'] = pd.merge(src_df['forms_raw4'], src_df['forms_raw4_2'][['label', 'name']], how='left', on='label')

In [8]:
src_df['forms_name'] = src_df['forms_name'].append(src_df['forms_raw4_2'], sort=False)

In [9]:
src_df['forms_name'] = pd.merge(src_df['forms_name']['unitkey'], src_df['forms_raw6'], how='left', on='unitkey')

In [10]:
src_df['forms_name'] = src_df['forms_name'].append(src_df['forms_raw6'], sort=False)

In [11]:
src_df['forms_name'] = src_df['forms_name'].drop_duplicates().sort_values('unitkey')

In [12]:
src_df['forms_name'].to_csv('forms_raw6_12.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)